# Robocorp Toolkit

此笔记本涵盖了如何开始使用 [Robocorp Action Server](https://github.com/robocorp/robocorp) action toolkit 和 LangChain。

Robocorp 是将 AI 代理、助手和副驾驶的功能扩展到自定义操作的最简单方法。

## 安装

首先，请参阅 [Robocorp Quickstart](https://github.com/robocorp/robocorp#quickstart)，了解如何设置 `Action Server` 和创建你的 Actions。

在你的 LangChain 应用程序中，安装 `langchain-robocorp` 包：

In [ ]:
# Install package
%pip install --upgrade --quiet langchain-robocorp

当你按照上面的快速入门创建新的 `Action Server` 时。

它将创建一个包含文件的目录，其中有 `action.py`。

我们可以添加 Python 函数作为 action，如[此处](https://github.com/robocorp/robocorp/tree/master/actions#describe-your-action)所示。

我们来为 `action.py` 添加一个虚拟函数。

```python
@action
def get_weather_forecast(city: str, days: int, scale: str = "celsius") -> str:
    """
    Returns weather conditions forecast for a given city.

    Args:
        city (str): Target city to get the weather conditions for
        days: How many day forecast to return
        scale (str): Temperature scale to use, should be one of "celsius" or "fahrenheit"

    Returns:
        str: The requested weather conditions forecast
    """
    return "75F and sunny :)"
```

然后我们启动服务器：

```bash
action-server start
```

然后我们就可以看到：

```
Found new action: get_weather_forecast

```

在本地测试，请访问正在运行的服务器 `http://localhost:8080`，并使用 UI 来运行该函数。

## 环境设置

您还可以选择设置以下环境变量：

- `LANGSMITH_TRACING=true`：启用 LangSmith 日志运行跟踪，该跟踪也可以绑定到相应的 Action Server 操作运行日志。更多信息请参阅 [LangSmith 文档](https://docs.smith.langchain.com/tracing#log-runs)。

## 用法

我们已经启动了本地操作服务器，如下所示，运行在 `http://localhost:8080`。

In [7]:
from langchain.agents import AgentExecutor, OpenAIFunctionsAgent
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI
from langchain_robocorp import ActionServerToolkit

# Initialize LLM chat model
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Initialize Action Server Toolkit
toolkit = ActionServerToolkit(url="http://localhost:8080", report_trace=True)
tools = toolkit.get_tools()

# Initialize Agent
system_message = SystemMessage(content="You are a helpful assistant")
prompt = OpenAIFunctionsAgent.create_prompt(system_message)
agent = OpenAIFunctionsAgent(llm=llm, prompt=prompt, tools=tools)

executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

executor.invoke("What is the current weather today in San Francisco in fahrenheit?")



> Entering new AgentExecutor chain...

Invoking: `robocorp_action_server_get_weather_forecast` with `{'city': 'San Francisco', 'days': 1, 'scale': 'fahrenheit'}`


"75F and sunny :)"The current weather today in San Francisco is 75F and sunny.

> Finished chain.


{'input': 'What is the current weather today in San Francisco in fahrenheit?',
 'output': 'The current weather today in San Francisco is 75F and sunny.'}

### 单输入工具

默认情况下，`toolkit.get_tools()` 将返回结构化工具形式的动作。

要返回单输入工具，请传递一个用于处理输入的 Chat 模型。

In [9]:
# Initialize single input Action Server Toolkit
toolkit = ActionServerToolkit(url="http://localhost:8080")
tools = toolkit.get_tools(llm=llm)